<a href="https://colab.research.google.com/github/KamilZPWr/Skyhack-3/blob/main/Patryk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install catalyst

In [22]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

from PIL import Image
from torch.utils.data import Dataset, DataLoader ,random_split
from torchvision import datasets, models, transforms

from catalyst import dl
from catalyst.dl.callbacks import AccuracyCallback, EarlyStoppingCallback
from catalyst.utils import metrics

In [4]:
ROOT_DIR = '/content/drive/My Drive/Colab Notebooks/'
NUM_CLASSES = 38
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(DEVICE)

cpu


In [24]:
class MultiClassDataset(Dataset):

    def __init__(self , csv_file , img_dir , transform=None):
        self.df = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
    
    def __getitem__(self,idx):
        d = self.df.iloc[idx]
        image = Image.open(f'{self.img_dir}/{d.Name}').convert("RGB")
        label = torch.tensor(d[1:].tolist() , dtype=torch.float32)
    
        if self.transform is not None:
            image = self.transform(image)
        return image, label
  
    def __len__(self):
        return len(self.df)


In [25]:
batch_size = 16
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
dataset = MultiClassDataset(ROOT_DIR + 'data/training_labels.csv' , ROOT_DIR + 'data/training_images' , transform)

number_of_validation_samples = int(len(dataset)*0.10) 
training_set, validation_set  = random_split(dataset , [len(dataset) - number_of_validation_samples, number_of_validation_samples])

print(f'Number of samples: Train: {len(training_set)}, Validation: {len(validation_set)}')
loaders = {
    "train":DataLoader(training_set , shuffle=True, batch_size=batch_size),
    "valid": DataLoader(validation_set , shuffle=True, batch_size=batch_size)
    }

Number of samples: Train: 3733, Validation: 414


In [89]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

def get_model(num_classes, feature_extract=True):
    model = models.alexnet(pretrained = True)
    set_parameter_requires_grad(model, feature_extract)
    num_ftrs = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(num_ftrs, num_classes)
    return model

def skyhacks_f1_score(preds, y):
    return np.mean([metrics.f1_score(preds[:, i], y[:, i])[0].item() for i in range(y_pred.shape[0])])

In [90]:
model = get_model(NUM_CLASSES)

In [91]:
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
callbacks = [
    EarlyStoppingCallback(patience=5)
]
num_epochs = 2

class CustomRunner(dl.Runner):
    
    def predict_batch(self, batch):
        # model inference step
        return self.model(batch[0].to(self.device).view(batch[0].size(0), -1))
    
    def _handle_batch(self, batch):
        x, y = batch
        output = self.model(x)
        loss = self.state.criterion(output, y)

        preds = torch.sigmoid(output).data > 0.5
        preds = preds.to(torch.float32)

        f1 = skyhacks_f1_score(preds, y)
        self.batch_metrics = {
            "loss": loss,
            "f1-score": f1
        }
        
        if self.state.is_train_loader:
            loss.backward()
            self.state.optimizer.step()
            self.state.optimizer.zero_grad()           


In [ ]:
runner = CustomRunner()

runner.train(
    loaders=loaders,
    model=model, 
    criterion=criterion, 
    optimizer=optimizer,
    callbacks=callbacks,
    num_epochs=num_epochs, 
    logdir="./logs", 
    verbose=False
)

tensor(0.7656, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
0.6185603522126197
tensor(0.4647, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
0.8707314683124423
tensor(0.3165, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
0.897798577323556
tensor(0.2512, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
0.9292372539639473
tensor(0.2162, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
0.9095694273710251
tensor(0.2887, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
0.9084953479468822
tensor(0.2662, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
0.9285161830484867
tensor(0.2807, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
0.9404622837901115
tensor(0.2157, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
0.9341632425785065
tensor(0.2567, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
0.9385812692344189
tensor(0.3271, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
0.9266222268342972
tensor(0.2071, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
0.9517350755631924
tensor(0.3830, gr